# Arrival Comparisons: NextBus Predictions vs. ATCS Observations

v.01 -- 6 February 2019 

This notebook contains the code required to read in NextBus arrival predictions and compare those predictions to logged observations from the ATCS system.

## 1. Initialization

In [1]:
import pandas as pd, numpy as np, requests, time, csv, datetime, re, os, zipfile, io
from datetime import datetime
pd.set_option("max_rows", 100)

import xml.etree.ElementTree

from matplotlib import pyplot as plt

from scipy import stats

The _day2read_ variable is entered by the user. It represents the day of revenue service that the user wishes to run the report for. The variable is entered in the following format: YYYY-MM-DD (e.g. 2019-01-24 for 24 January 2019).

If the user enters 2019-01-24, all NextBus arrival prediction records from 3 AM on 2019-01-24 until 2:59 AM on 2019-01-25 are returned.

In [2]:
day2read = input("select day (YYYY-MM-DD): ")

select day (YYYY-MM-DD): 2019-01-24


In [3]:
# function to keep lists of predictions, train numbers the same length
# useful for adding these lists to DataFrames

def addNaN(list2pass, list_length):
    
    list2pass = sorted(list2pass)
    list2pass = list2pass[:list_length]
    
    
    while len(list2pass) < list_length:
        list2pass += [np.NaN]
    return list2pass

def toReadableTime(ts):
    return datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S').split()[1]
def toReadableDateTime(ts):
    return datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

def dayIncrement(readYear, readMonth, readDay):
    # 30-day months
    if readMonth in ["04","06","09","11"]:
        if readDay == "30":
            nextDate = readYear + "-" + str(int(readMonth) + 1).zfill(2) + "-01"
        elif int(readDay) > 30:
            print("INVALID DATE")
            nextDate = "INVALID DATE"
        else:
            nextDate = readYear + "-" + readMonth + "-" + str(int(readDay) + 1).zfill(2)
    # 31-day months
    elif readMonth in ["01","03","05","07","08","10"]:
        if readDay == "31":
            nextDate = readYear + "-" + str(int(readMonth) + 1).zfill(2) + "-01"
        elif int(readDay) > 31:
            print("INVALID DATE")
            nextDate = "INVALID DATE"
        else:
            nextDate = readYear + "-" + readMonth + "-" + str(int(readDay) + 1).zfill(2)
    # December
    elif readMonth == "12":
        if readDay == "31":
            nextDate = str(int(readYear) + 1) + "-" + "01-01"
        elif int(readDay) > 31:
            print("INVALID DATE")
            nextDate = "INVALID DATE"
        else:
            nextDate = readYear + "-" + readMonth + "-" + str(int(readDay) + 1).zfill(2)
    # February
    elif readMonth == "02":
        if int(readYear) % 4 != 0:
            if readDay == "28":
                nextDate = readYear + "-" + str(int(readMonth) + 1).zfill(2) + "-01"
            elif int(readDay) > 31:
                print("INVALID DATE")
                nextDate = "INVALID DATE"
            else:
                nextDate = readYear + "-" + readMonth + "-" + str(int(readDay) + 1).zfill(2)
        else:
            if readDay == "29":
                nextDate = readYear + "-" + str(int(readMonth) + 1).zfill(2) + "-01"
            elif int(readDay) > 31:
                print("INVALID DATE")
                nextDate = "INVALID DATE"
            else:
                nextDate = readYear + "-" + readMonth + "-" + str(int(readDay) + 1).zfill(2)
    else:
        pass
    return nextDate

def consist_to_four_digit(consist):
    if pd.isnull(consist) == False and len(str(consist))>0:
        if type(consist) == str:
            consist = int(consist)
            if consist >= 1 and consist <= 400:
                fourdigitconsist = consist + 1199
            elif consist >= 501 and consist <= 999:
                fourdigitconsist = consist + 1499
            else:
                fourdigitconsist = int(consist)

        else:
            consist = int(consist)
            if consist >= 1 and consist <= 400:
                fourdigitconsist = consist + 1199
            elif consist >= 501 and consist <= 999:
                fourdigitconsist = consist + 1499
            else:
                fourdigitconsist = int(consist)
    else:
        fourdigitconsist = np.NaN
    return fourdigitconsist

def loadSMC(day2read):
    # getfile = "http://nbsnap/atcslogs/SmcSls/%sSLS.smc" % str(datetime.datetime.today()).split()[0].replace("-","")[2:]
    # day2read should be in the format YYYY-MM-DD
    
    readYear = day2read.split("-")[0]
    readMonth= day2read.split("-")[1]
    readDay= day2read.split("-")[2]
    
    getfile = "http://nbsnap/atcslogs/SmcSls/{}SLS.smc".format((readYear[2:]+readMonth+readDay))
    
    readMsg = "Running for {}-{}-{}".format(readYear, readMonth, readDay)
    print(readMsg)
    s = requests.get(getfile)
    print(s)
    
    if s.status_code == 200:
        # if the SMC log hasn't been archived
        smc = s.text
        smcrecords = smc.replace("\n","").split("\r")
        archived = False
    else:
        # if the SMC log has already been archived
        print("File not found. Requesting from archive...")
        getfile = "http://nbsnap/atcslogs/SmcSls/Archive/{}SLSsmc.zip".format((readYear[2:]+readMonth+readDay))
        r = requests.get(getfile)
        print(r, "from archive")
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall()
        
        localpath = "{}SLS.smc".format((readYear[2:]+readMonth+readDay))
        smcobject = open(localpath, "r")
        smclist = []


        for x in smcobject:
            smclist.append(x)

        smcrecords = [x.replace("\n", "").strip() for x in smclist]

        del smclist
        archived = True
        
    #smc = s.text
    #smcrecords = smc.replace("\n","").split("\r")

    # remove records we don't want, so future operations are faster
    poplist = []
    couples = []

    print(len(smcrecords), "initial records")

    for i in range(len(smcrecords)):
        rec = smcrecords[i].replace("{", "").replace("}","").split()

        if "Arrived" in rec and "Destination" in rec and "Platform" in rec:
            pass
        elif "Vehicle" in rec and "IDs" in rec:
            couples.append(smcrecords[i])
            poplist.append(i)
        else:    
            poplist.append(i)

    for item in reversed(poplist):
        smcrecords.pop(item)

    print(len(smcrecords), "arrival records")
    
    if archived == False:
        smcrecords = smc.replace("\n","").split("\r")
    else:
        smcobject = open(localpath, "r")
        smclist = []
        for x in smcobject:
            smclist.append(x)
        smcrecords = [x.replace("\n", "").strip() for x in smclist]
        del smclist
        os.remove(localpath)
        
    coupleDict = {}

    for couple in couples:

        info = couple.replace("{", "").replace("}","").split()

        veh1 = consist_to_four_digit(info[6])
        veh2 = consist_to_four_digit(info[7])

        coupleDict[veh1] = veh2

    # remove records we don't want, so future operations are faster
    poplist = []
    couples = []

    for i in range(len(smcrecords)):
        rec = smcrecords[i].replace("{", "").replace("}","").split()

        if "Arrived" in rec and "Destination" in rec and "Platform" in rec:
            pass
        elif "Vehicle" in rec and "IDs" in rec:
            couples.append(smcrecords[i])
            poplist.append(i)
        else:    
            poplist.append(i)

    for item in reversed(poplist):
        smcrecords.pop(item)

    dfs = []

    for record in smcrecords:
        rec = [x.replace("{","").replace("}","") for x in record.split(" ",2)]
        dfs.append(pd.DataFrame([rec], columns=["TIME", "SRS_ID", "MSG"]))

    smcdf = pd.concat(dfs, ignore_index=True)

    smcdf["CONSIST"] = [re.sub("\D", "", x.lower().split()[x.lower().split().index('consist')+1]) if "consist" in x.lower().split() else '' for x in smcdf['MSG']]

    smcdf["FOUR_DIGIT_CONSIST"] = [consist_to_four_digit(x) for x in smcdf['CONSIST']]
    smcdf["PLATFORM"] = [x.split()[x.split().index("Platform")+1] for x in smcdf["MSG"]]

    smcdf["H"] = [int(x.split(":")[0]) for x in smcdf["TIME"]]
    dayChangeIdx = smcdf[smcdf["H"].diff() < 0].index[0]
    smcdf.drop("H", axis=1, inplace=True)

    smcdf["DAY"] = np.NaN
    smcdf.loc[:(dayChangeIdx-1), "DAY"] = day2read
    smcdf.loc[dayChangeIdx:, "DAY"] = dayIncrement(readYear=readYear, readMonth=readMonth, readDay=readDay)

    smcdf["DATETIME"] = smcdf["DAY"] + " " + smcdf["TIME"]


    #smcdf["DATETIME"] = [pd.to_datetime("2019-01-16 " +x) for x in smcdf["TIME"]]
    smcdf["UNIX"] = [(time.mktime(time.strptime(str(x).split(".")[0], "%Y-%m-%d %H:%M:%S"))) for x in smcdf["DATETIME"]]

    return smcdf, coupleDict

def generateUniqueId(uid_no):
    return str(day2read + "--" + str(uid_no))

def loadSMCforLine(day2read):
    # getfile = "http://nbsnap/atcslogs/SmcSls/%sSLS.smc" % str(datetime.datetime.today()).split()[0].replace("-","")[2:]
    # day2read should be in the format YYYY-MM-DD
    
    readYear = day2read.split("-")[0]
    readMonth= day2read.split("-")[1]
    readDay= day2read.split("-")[2]
    
    getfile = "http://nbsnap/atcslogs/SmcSls/{}SLS.smc".format((readYear[2:]+readMonth+readDay))
    
    readMsg = "Running for {}-{}-{}".format(readYear, readMonth, readDay)
    print(readMsg)
    s = requests.get(getfile)
    print(s)
    
    if s.status_code == 200:
        # if the SMC log hasn't been archived
        smc = s.text
        smcrecords = smc.replace("\n","").split("\r")
    else:
        # if the SMC log has already been archived
        print("File not found. Requesting from archive...")
        getfile = "http://nbsnap/atcslogs/SmcSls/Archive/{}SLSsmc.zip".format((readYear[2:]+readMonth+readDay))
        r = requests.get(getfile)
        print(r, "from archive")
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall()
        localpath = "{}SLS.smc".format((readYear[2:]+readMonth+readDay))
        smcobject = open(localpath, "r")
        smclist = []
        for x in smcobject:
            smclist.append(x)
        smcrecords = [x.replace("\n", "").strip() for x in smclist]
        del smclist
        os.remove(localpath)
    return smcrecords

## Notebook Contents
1. Initialization
2. Load NextBus data
3. Load ATCS arrival data
4. Load ATCS signs data
5. Match NB 2 ATCS
6. Match ATCS 2 NB
7. Report

## 2. Load NextBus Data

This section of the notebook loads data collected from the NextBus API into memory. The collected data is currently stored in the _output/_ directory, and if it is moved, the filepaths will need to be adjusted accordingly.

To accurately capture an entire day of revenue service, the loading of the data has been expanded to include early morning predictions of the following day.

In [4]:
filepath = "output/NB-VNR-{}.csv".format(day2read)
nextDay2Read = dayIncrement(day2read.split("-")[0], day2read.split("-")[1], day2read.split("-")[2])
filepathPastMidnight = "output/NB-VNR-{}.csv".format(nextDay2Read)

data = pd.read_csv(filepath)

# remove records from before 3 AM; those records belong to the previous day of revenue of service
data = data[data["Timestamp"] >= datetime.strptime("{} 11:00:00".format(day2read),'%Y-%m-%d %H:%M:%S' ).timestamp()]

try:
    # if there is data available for the following calendar day, add the arrivals belonging to the revenue service of 
    # the day entered as day2read
    dataPastMidnight = pd.read_csv(filepathPastMidnight)
    ThreeAM = datetime.strptime("{} 11:00:00".format(nextDay2Read),'%Y-%m-%d %H:%M:%S' ).timestamp()
    dataPastMidnight = dataPastMidnight[dataPastMidnight["Timestamp"] < ThreeAM]
    data = data.append(dataPastMidnight, ignore_index=True)
    
except OSError:
    # if there is no data for the day, print a warning message and continue
    onlyOneDayErrorMsg = "Only one calendar day of data is available. Analysis excludes arrivals made after 23:59 on {}".format(day2read)
    print(onlyOneDayErrorMsg)

# Reset index for newly appended DataFrame
data.reset_index(inplace=True,drop=True)
sample = data.query("Seconds < 600")

The following cell assings unique IDs to each vehicle arrival. This step is necessary because vehicle ID is not unique; the same vehicle arrives at a platform several times over the course of a revenue day. By assigning unique arrival IDs, we can match individual arrivals across data sources. 

In [5]:
%%time

vehicles = list(data["Vehicle"].unique())

print(len(vehicles), "unique vehicles.")

arrivals = pd.DataFrame(columns=["Timestamp", "Line", "Vehicle", "Seconds", "Diff", "Cut-Off", "UID"])

uid_no = 1
for vehicle in vehicles:
    
    testtrain = data[data["Vehicle"] == vehicle]
    testtrain["Diff"] = testtrain["Timestamp"].diff()
    testtrain["Cut-Off"] = testtrain["Diff"] >= (45*60)
    o = 0

    for i in list(testtrain[testtrain["Cut-Off"]].index) + [testtrain.index.max()]:
        #unique_train = generateUniqueId(uid_no)
        unique_train = "UID-"+str(uid_no)
        #print(unique_train)
        #print(testtrain.loc[o:i-1])

        try:
            #ii = testtrain["Seconds"].loc[o:i-1].idxmin()
            ii = testtrain["Seconds"].loc[o:i].idxmin()
            values = list(testtrain.loc[ii]) + [unique_train]
            #print(values)
            
            df2 = pd.DataFrame([values], columns=["Timestamp", "Line", "Vehicle", "Seconds", "Diff", "Cut-Off", "UID"])
            arrivals = arrivals.append(df2, ignore_index=True)
            
        except:
            pass
            #print(vehicle)
        uid_no += 1
        o = i
arrivals = arrivals.sort_values("Timestamp")[["Timestamp", "Line", "Vehicle", "Seconds", "UID"]].reset_index(drop=True)

print(len(vehicles), "unique vehicles.", "--", arrivals["UID"].nunique(), "unique arrival IDs")

arrivals["Arrival Timestamp"] = arrivals["Timestamp"] + arrivals["Seconds"]
arrivals["Arrival Time"] = [toReadableTime(x) for x in arrivals["Arrival Timestamp"]]

arrivals = arrivals.sort_values("Arrival Timestamp")

115 unique vehicles.


/home/pynotebook/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/pynotebook/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


115 unique vehicles. -- 348 unique arrival IDs
CPU times: user 16.3 s, sys: 6 ms, total: 16.3 s
Wall time: 16.3 s


## 3. Load ATCS Arrival Data

This section of the notebook reads SMC data logs off of the Muni Central Control webpage and converts the relevant components of those logs into Pandas DataFrames.

In [6]:
%%time
smc, coupleDict = loadSMC(day2read=day2read)

Running for 2019-01-24
<Response [404]>
File not found. Requesting from archive...
<Response [200]> from archive
492146 initial records
8870 arrival records
CPU times: user 14.1 s, sys: 287 ms, total: 14.4 s
Wall time: 14.5 s


SMC arrival log DataFrame is further filtered below to only include arrivals at the relevant platform and within the timeframe covered by revenue service and for which there is NextBus predicted arrival data.

In [7]:
vnr = smc[smc["PLATFORM"] == "VNR"].reset_index(drop=True)

vnr = vnr[["DATETIME","TIME", "UNIX", "FOUR_DIGIT_CONSIST"]]
vnr["SECOND_CONSIST"] = [coupleDict[x] for x in vnr["FOUR_DIGIT_CONSIST"]]
print(len(vnr), "arrivals at VNR")

t_min = data['Timestamp'].min() - 360
t_max = data["Timestamp"].max() + 360

vnr_t = vnr[(vnr["UNIX"] > t_min) & (vnr["UNIX"] < t_max)]
print(len(vnr_t), "arrivals at VNR within timeframe")

vnr_t["DIFF"] = vnr_t["UNIX"].diff()

dropList = []

atcs_dupes_list = list(vnr_t[vnr_t["DIFF"] <= 60].index)

for dupeIdx in atcs_dupes_list:
    if vnr_t.loc[dupeIdx, "FOUR_DIGIT_CONSIST"] == vnr_t.loc[(dupeIdx-1), "FOUR_DIGIT_CONSIST"] or vnr_t.loc[dupeIdx, "FOUR_DIGIT_CONSIST"] == vnr_t.loc[(dupeIdx-1), "SECOND_CONSIST"]:
        dropList.append(dupeIdx)
        
vnr_t.drop(dropList, axis=0, inplace=True)
print(len(vnr_t), "de-duplicated arrivals within timeframe")

558 arrivals at VNR
311 arrivals at VNR within timeframe
310 de-duplicated arrivals within timeframe


/home/pynotebook/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/pynotebook/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
test_vnr_t = vnr_t.copy(deep=True)

test_vnr_t["DIFF"] = test_vnr_t["UNIX"].diff()

In [9]:
dropList = []

atcs_dupes_list = list(test_vnr_t[test_vnr_t["DIFF"] <= 60].index)

for dupeIdx in atcs_dupes_list:
    if test_vnr_t.loc[dupeIdx, "FOUR_DIGIT_CONSIST"] == test_vnr_t.loc[dupeIdx-1, "FOUR_DIGIT_CONSIST"] or test_vnr_t.loc[dupeIdx, "FOUR_DIGIT_CONSIST"] == test_vnr_t.loc[dupeIdx-1, "SECOND_CONSIST"]:
        dropList.append(dupeIdx)
        
test_vnr_t.drop(dropList, axis=0, inplace=True)

In [10]:
test_vnr_t.loc[sorted([x-1 for x in atcs_dupes_list] + atcs_dupes_list)].drop_duplicates()

,DATETIME,TIME,UNIX,FOUR_DIGIT_CONSIST,SECOND_CONSIST,DIFF
292,2019-01-24 14:45:41,14:45:41,1.548370e+09,1481,1545,76.0
293,2019-01-24 14:46:40,14:46:40,1.548370e+09,1404,1547,59.0
296,2019-01-24 14:51:49,14:51:49,1.548370e+09,1469,0,170.0
297,2019-01-24 14:52:46,14:52:46,1.548370e+09,1500,1447,57.0
358,2019-01-24 16:35:12,16:35:12,1.548377e+09,1415,0,65.0
359,2019-01-24 16:36:08,16:36:08,1.548377e+09,1459,1458,56.0
457,2019-01-24 19:31:11,19:31:11,1.548387e+09,1426,0,141.0
458,2019-01-24 19:32:07,19:32:07,1.548387e+09,1510,1432,56.0
468,2019-01-24 20:02:22,20:02:22,1.548389e+09,1459,1458,64.0
469,2019-01-24 20:03:21,20:03:21,1.548389e+09,2037,2004,59.0


## 4. Load ATCS Sign Data

The following section assigns lines to ATCS arrival records by reading in ATCS logs describing the setting of signs on the light-rail vehicles. A Pandas DataFrame is generated that provides every record of a sign assignment or change during the revenue day; these records can be used later to attribute ATCS arrivals with the appropriate line.

In [11]:
signcodes = [
    ["8","J","OB"],
    ["9","J","OB"],
    ["10","J","OB"],
    ["11","J","OB"],
    ["12","J","OB"],
    ["13","J","OB"],
    ["14","J","IB"],
    ["15","J","IB"],
    ["30","K","OB"],
    ["32","K","OB"],
    ["33","K","IB"],
    ["38","K","OB"],
    ["55", "L", "OB"],
    ["56","L","OB"],
    ["57","L", "OB"],
    ["58","L","OB"],
    ["60", "L","OB"],
    ["61","L","IB"],
    ["62","L","IB"],
    ["77","M","OB"],
    ["80","M","OB"],
    ["81","M","IB"],
    ['82',"M","IB"],
    ["87","M","OB"],
    ["91","M","OB"],
    ["100","N","OB"],
    ["103","N","OB"],
    ["107","N","IB"],
    ["108","N","OB"],
    ["109","N","IB"],
    ["115","N","IB"],
    ["119","N","OB"],
    ["124","N","OB"],
    ["153","T","OB"],
    ["161","T","IB"],
    ["169","T","IB"],
    ["175","T","IB"],
    ["134","S","OB"],
    ["135","S","OB"],
    ["136","S","IB"],
    ["215","S","IB"],
    ["218","S","OB"],
    ["137","S","IB"],
    ["227","X",np.NaN]
]

signcodesDF = pd.DataFrame(data=signcodes, columns=["SIGN CODE", "LINE", "DIRECTION"])

In [12]:
smcrecords = loadSMCforLine(day2read=day2read)

Running for 2019-01-24
<Response [404]>
File not found. Requesting from archive...
<Response [200]> from archive


In [13]:
%%time



consist2signList = []

for rec in smcrecords:
    rec = rec.replace("{","").replace("}","")
    rectime = rec.split(" ")[0]
    if "Sign" in rec:
        try:
            consist = rec.split()[rec.split().index("Consist")+1]
            if "Sign" in rec.split():
                sign = rec.split()[rec.split().index("Sign")+3]
            elif "Signs" in rec.split():
                sign = rec.split()[rec.split().index("Signs")+1]
            else:
                pass

            consist2signList.append(pd.DataFrame(data=[[rectime, consist_to_four_digit(consist), sign]], columns=["TIME","CONSIST", "SIGN CODE"]))
        except ValueError:
            pass
    else:
        pass
#consist2sign = pd.concat(consist2signList, ignore_index=True).drop_duplicates(subset=["CONSIST"], keep="last").merge(signcodesDF, on="SIGN CODE")
signs = pd.concat(consist2signList, ignore_index=True)

#day2read = "2019-01-16"
readYear = day2read.split("-")[0]
readMonth = day2read.split("-")[1]
readDay = day2read.split("-")[2]

signs["H"] = [int(x.split(":")[0]) for x in signs["TIME"]]
dayChangeIdx = signs[signs["H"].diff() < 0].index[0]
signs.drop("H", axis=1, inplace=True)

signs["DAY"] = np.NaN
signs.loc[:(dayChangeIdx-1), "DAY"] = day2read
signs.loc[dayChangeIdx:, "DAY"] = dayIncrement(readYear=readYear, readMonth=readMonth, readDay=readDay)



signs["DATETIME"] = signs["DAY"] + " " + signs["TIME"]

signs.drop(["DAY", "TIME"], axis=1, inplace=True)

# remove sign codes that are not found in the reference
signs = signs[signs["SIGN CODE"].isin(signcodesDF["SIGN CODE"].unique())]

# drop duplicate entries
signs = signs.drop_duplicates(subset=["CONSIST", "DATETIME", "SIGN CODE"])
signs["UNIX"] = [(time.mktime(time.strptime(str(x).split(".")[0], "%Y-%m-%d %H:%M:%S"))) for x in signs["DATETIME"]]

signs2consists = signs.merge(signcodesDF, on="SIGN CODE")

CPU times: user 32.3 s, sys: 279 ms, total: 32.5 s
Wall time: 32.4 s


## 5. Match NextBus Arrivals to ATCS Arrivals

This section of the notebook involves the code for evaluating each NextBus predicted arrival against the ATCS observation and determining if that NextBus predicted arrival can be "matched" to a single ATCS observation. A typology of matches is created (_matchCodes_), and each match event (either a match or not a match) is classified into a typology depending on how the match was or was note made.

In [14]:
matchCodes = {
    1 : "MATCHED ON VEHICLE ID",
    2 : "DID NOT MATCH; GHOST TRAIN; NO UN-MATCHED ARRIVALS",
    3 : "MATCHED; ONE REMAINING MATCH; SAME LINE",
    4 : "MATCHED TO OUT-OF-SERVICE TRAIN",
    5 : "DID NOT MATCH; ATCS SHOWS DIFFERENT LINE THAN NEXTBUS",
    6 : "DID NOT MATCH; MULTIPLE OUT-OF-SERVICE TRAINS",
    7 : "MATCHED; ONE UN-MATCHED RECORD HAS SAME LINE",
    8 : "DID NOT MATCH; ATCS SHOWS MULTIPLE POSSIBLE TRAINS; SAME LINE",
    9 : "DID NOT MATCH; GHOST TRAIN; NO ATCS RECORDS IN TIME-WINDOW",
    10 : "DID NOT MATCH; ATCS SHOWS MULTIPLE POSSIBLE TRAINS; DIFFERENT LINES",
    11 : "MATCHED TO AN ALREADY MATCHED ATCS OBSERVATION"
}

matchDescriptions = {
    "MATCHED ON VEHICLE ID" : "The vehicle ID is present in both the NextBus and ATCS datasets within the given time window.",
    "DID NOT MATCH; GHOST TRAIN; NO UN-MATCHED ARRIVALS" : "There are no un-matched arrivals in the ATCS dataset within the given time window; all ATCS records within that time window are matched to other NextBus arrivals.",
    "MATCHED; ONE REMAINING MATCH; SAME LINE" : "There is one remaining un-matched arrival in the ATCS dataset within the given time window, and it is the same line as the NextBus arrival.",
    "MATCHED TO OUT-OF-SERVICE TRAIN" : "There is one remaining un-matched arrival in the ATCS dataset within the given time window, and it shown as an out-of-service train.",
    "DID NOT MATCH; ATCS SHOWS DIFFERENT LINE THAN NEXTBUS" : "There is one remaining un-matched arrival in the ATCS dataset within the given time window, and it is a different line than the NextBus arrival.",
    "DID NOT MATCH; MULTIPLE OUT-OF-SERVICE TRAINS" : "There are multiple remaining un-matched arrivals in the ATCS dataset within the given time window, and they are shown as out-of-service trains",
    "MATCHED; ONE UN-MATCHED RECORD HAS SAME LINE" : "There is one remaining un-matched arrival in the ATCS dataset within the given time window, and it is the same line as the NextBus arrival.",
    "DID NOT MATCH; ATCS SHOWS MULTIPLE POSSIBLE TRAINS; SAME LINE" : "There are multiple remaining un-matched arrivals in the ATCS dataset within the given time window, and more than one is the same line as the NextBus arrival",
    "DID NOT MATCH; GHOST TRAIN; NO ATCS RECORDS IN TIME-WINDOW" : "There are no ATCS arrivals within the given time window.",
    "DID NOT MATCH; ATCS SHOWS MULTIPLE POSSIBLE TRAINS; DIFFERENT LINES" : "There are multiple remaining un-matched arrivals in the ATCS dataset within the given time window, and they are different lines than the NextBus arrival.",
    "MATCHED TO AN ALREADY MATCHED ATCS OBSERVATION" : "NextBus treated each car in a two-car consist as a separate arrival, thereby counting twice in NextBus but only once in ATCS."
}

### Matcher
The matcher iterates through all NextBus arrival predictions and attempts to match each prediction to a single observation from the arrivals in the ATCS data logs. Each NextBus arrival prediction advances through a sequence of if-else statements until either a match is found, or it is determined that a match cannot be found.

When a match is determined to have been found, the NextBus arrival prediction is removed from the DataFrame _unmatchedNBarrivals_, and the ATCS observation with which it has been determined to have matched is removed from the DataFrame _unmatchedATCSarrivals_, thereby creating two up-to-date datasets of arrivals for which there are no matches and removing the possibility that multiple NextBus arrivals could be considered to have matched with a single ATCS observation, for example. 

In [15]:

# array containing difference in times for matching records
# used for analysis on the differences between datasets
matchTimeDif = []

# DataFrame of matches
matchDFcolumns = ["NextBus Time", "NextBus Vehicle", "NextBus Line", "ATCS Time", "ATCS Vehicle", "ATCS Line", "Match Type"]
matchDF = pd.DataFrame(columns=matchDFcolumns)

# DataFrames for storing unmatched arrival records
unmatchedNBarrivals = arrivals[arrivals["Seconds"] <= 60].copy(deep=True)
unmatchedATCSarrivals = vnr_t.copy(deep=True)

# DataFrame to store the matching events typologies for future summary statistics used in the exported report
arrivalMatchingCols = ["NextBus Vehicle ID", "NextBus Line", "NextBus Arrival Time", "Match Code", "Match Code Description", "Alternatives Matrix"]
arrivalMatching = pd.DataFrame(columns=arrivalMatchingCols)


numArrivals = len(arrivals[arrivals["Seconds"] <= 60].index)
matches = 0
confirmedGhosts = 0
matchedViaRemovalLine = 0

# this value is the radius (in seconds) of the time buffer used to search for eligible matches
# a timeRadius of 180 means that, for every NextBus arrival prediction, ATCS observations for
# three minutes after that NextBus arrival prediction are considered as potential matches. 
timeRadius = 240 # minutes

for i in arrivals[arrivals["Seconds"] <= 60].index:
    # generate time window
    arrivalTime = arrivals.loc[i, "Timestamp"]
    arrivalVehID = arrivals.loc[i, "Vehicle"]
    arrivalLine = arrivals.loc[i, "Line"]
    arrivalReadableTime = arrivals.loc[i, "Arrival Time"]
    t_max = arrivalTime + timeRadius
    t_min = arrivalTime - 60 # reconfigured the window to make it asymmetrical, so it looks past the the NextBus predicted arrival by 4 min, but before it by only 1 min
    
    # this matrix stores potential matches if more than one is found
    altMatrix = np.NaN
    
    # generate DFs of only arrivals within the time window across both datasets
    searchDFarrivals = unmatchedATCSarrivals[(unmatchedATCSarrivals["UNIX"] < t_max) & (unmatchedATCSarrivals["UNIX"] > t_min)]
    arrivalDFarrivals = arrivals[(arrivals["Timestamp"] < t_max) & (arrivals["Timestamp"] > t_min)]
    
    # ASSIGN LINE INFORMATION TO ATCS LOGS
    if len(searchDFarrivals) > 0:
        
        dfs = []

        for m in searchDFarrivals.index:
            car1 = searchDFarrivals.loc[m, "FOUR_DIGIT_CONSIST"]
            unix = searchDFarrivals.loc[m, "UNIX"]

            #print(car1, car2)

            tempDF = signs2consists[(signs2consists["CONSIST"].isin([car1])) & (signs2consists["UNIX"] <= unix) & (signs2consists["UNIX"] >= (unix - (60*60)))].sort_values("UNIX")

            if len(tempDF) == 0:
                line = "X"
            else:
                line = tempDF.loc[tempDF["UNIX"].idxmax(), "LINE"]
            dfs.append(pd.DataFrame([[car1, line]], columns=["FOUR_DIGIT_CONSIST", "LINE"]))

        try:
            lineAssignments = pd.concat(dfs, ignore_index=True)
        except ValueError:
            print(len(searchDFarrivals))

        searchDFarrivals = searchDFarrivals.merge(lineAssignments, on="FOUR_DIGIT_CONSIST")
        
        # OK, now that the arrivals dataset (NB) is loaded and subset, and the search dataset (ATCS) is both subset and has lines assigned
        # we can proceed
        
        
        
        if arrivalVehID in list(searchDFarrivals["FOUR_DIGIT_CONSIST"]):
            
            # OPTION 1 -- MATCHED ON VEHICLE ID 
            # the vehicle ID is present in both datasets, thereby allowing us to make the match.
            # This NextBus arrival is therefore considered to be MATCHED.
            matchCodeID = 1
            matches += 1
            
            # the following code determines the index of the ATCS observation with which the NextBus arrival prediction has been matched
            # so that the ATCS observation can be dropped from the unmatchedATCSarrivals DataFrame
            sdf_idx = list(searchDFarrivals["FOUR_DIGIT_CONSIST"]).index(arrivalVehID)
            datetime_value = searchDFarrivals["DATETIME"][sdf_idx]
            atcs_idx = unmatchedATCSarrivals[(unmatchedATCSarrivals["DATETIME"] == datetime_value) & (unmatchedATCSarrivals["FOUR_DIGIT_CONSIST"] == arrivalVehID)].index[0]
            
            # determine attributes of matched ATCS arrival
            matchTime = vnr_t.loc[atcs_idx, "TIME"]
            
            if vnr_t.loc[atcs_idx, "SECOND_CONSIST"] == 0:
                matchVeh = str(vnr_t.loc[atcs_idx, "FOUR_DIGIT_CONSIST"])
            else:
                matchVeh = str(vnr_t.loc[atcs_idx, "FOUR_DIGIT_CONSIST"]) + "/" + str(vnr_t.loc[atcs_idx, "SECOND_CONSIST"])
            
            matchLine = signs2consists[(signs2consists["CONSIST"] == vnr_t.loc[atcs_idx, "FOUR_DIGIT_CONSIST"]) & (signs2consists["UNIX"] < vnr_t.loc[atcs_idx, "UNIX"])].sort_values("UNIX").reset_index().tail(1)["LINE"].iloc[0]
            
            if matchLine in ["K","T"]:
                matchLine = "KT"
            else:
                pass
            
            matchPairs = [arrivalReadableTime, arrivalVehID, arrivalLine,matchTime,matchVeh,matchLine,matchCodes[matchCodeID]]
            matchDF = matchDF.append(pd.DataFrame([matchPairs], columns=matchDFcolumns), ignore_index=True)
            
            # the difference (in seconds) between the NextBus predicted arrival and the ATCS observation with which it was matched is added to this array for future analysis
            matchTimeDif.append(arrivalTime-unmatchedATCSarrivals.loc[atcs_idx, "UNIX"])
            
            # after the match is determined to have occurred, the NextBus arrival prediction is dropped from the unmatchedNBarrivals DataFrame
            unmatchedNBarrivals.drop(i, axis=0, inplace=True)
            unmatchedATCSarrivals.drop(atcs_idx, axis=0, inplace=True)
            
            
        elif arrivalVehID in list(searchDFarrivals["SECOND_CONSIST"]):
            
            # OPTION 1 -- MATCHED ON VEHICLE ID 
            # the vehicle ID is present in both datasets, thereby allowing us to make the match.
            # This NextBus arrival is therefore considered to be MATCHED.
            matchCodeID = 1
            matches += 1
            
            sdf_idx = list(searchDFarrivals["SECOND_CONSIST"]).index(arrivalVehID)
            datetime_value = searchDFarrivals["DATETIME"][sdf_idx]
            atcs_idx = unmatchedATCSarrivals[(unmatchedATCSarrivals["DATETIME"] == datetime_value) & (unmatchedATCSarrivals["SECOND_CONSIST"] == arrivalVehID)].index[0]
            
             # determine attributes of matched ATCS arrival
            matchTime = vnr_t.loc[atcs_idx, "TIME"]
            
            if vnr_t.loc[atcs_idx, "SECOND_CONSIST"] == 0:
                matchVeh = str(vnr_t.loc[atcs_idx, "FOUR_DIGIT_CONSIST"])
            else:
                matchVeh = str(vnr_t.loc[atcs_idx, "FOUR_DIGIT_CONSIST"]) + "/" + str(vnr_t.loc[atcs_idx, "SECOND_CONSIST"])
            
            matchLine = signs2consists[(signs2consists["CONSIST"] == vnr_t.loc[atcs_idx, "FOUR_DIGIT_CONSIST"]) & (signs2consists["UNIX"] < vnr_t.loc[atcs_idx, "UNIX"])].sort_values("UNIX").reset_index().tail(1)["LINE"].iloc[0]
            
            if matchLine in ["K","T"]:
                matchLine = "KT"
            else:
                pass
            
            matchPairs = [arrivalReadableTime, arrivalVehID, arrivalLine,matchTime,matchVeh,matchLine,matchCodes[matchCodeID]]
            matchDF = matchDF.append(pd.DataFrame([matchPairs], columns=matchDFcolumns), ignore_index=True)
            
            matchTimeDif.append(arrivalTime -unmatchedATCSarrivals.loc[atcs_idx, "UNIX"])
            
            unmatchedNBarrivals.drop(i, axis=0, inplace=True)
            unmatchedATCSarrivals.drop(atcs_idx, axis=0, inplace=True)
            
            
        # If the vehicle ID is not found in the search dataset, we can take several actions.    
        else:
            
            # create an array of vehicle IDs that are present in both the substs of the arrivals dataset and the search dataset.
            # these records can be considered to be MATCHED, thereby removing them from our consideration
            otherMatches = []
            for veh in arrivalDFarrivals["Vehicle"].unique():

                if veh in list(searchDFarrivals["FOUR_DIGIT_CONSIST"]):
                    otherMatches.append(veh)
                elif veh in list(searchDFarrivals["SECOND_CONSIST"]):
                    otherMatches.append(veh)
                else:
                    pass
            
            # the resultant DF is all of the arrivals within the subset search dataset for which there are no matching vehicle
            # ID matches in the subset arrivals dataset
            
            searchArrivals = searchDFarrivals[searchDFarrivals["FOUR_DIGIT_CONSIST"].isin(otherMatches) == False]
            altMatrix = np.matrix(searchArrivals)
            
            if len(searchArrivals) == 0:
                
                # OPTION 2 -- GHOST TRAIN; NO UN-MATCHED ARRIVALS
                # there are no un-matched arrivals in the subset search dataset
                # This NextBus arrival is therefore considered to be a GHOST TRAIN.
                matchCodeID = 2
                confirmedGhosts += 1
                print(arrivalVehID, "GHOST TRAIN -- no remaining, un-matched arrival in time window.")
            
            # if there is only one train in the remaining, un-matched subset of the search dataset, a number of outcomes can occur 
            elif len(searchArrivals) == 1:
                
                
                if arrivalLine == searchArrivals.iloc[0]["LINE"]:
                    
                    # OPTION 3 -- ONE REMAINING MATCH; SAME LINES
                    # the remaining, un-matched train in the subset search dataset is of the same line as the NextBus
                    # arrival being investigated, thereby allowing us to assert that the NextBus arrival matches this lone remaining
                    # un-matched ATCS arrival This NextBus arrival is therefore considered to be MATCHED.
                    
                    print(arrivalVehID, "MATCHED -- Matched via removal & line.")
                    matchCodeID = 3
                    matches += 1
                    matchedViaRemovalLine += 1
                    #matchedNBidx.append(i)
                    unmatchedNBarrivals.drop(i, axis=0, inplace=True)
                    
                    sdf_idx = 0
                    datetime_value = searchArrivals["DATETIME"][sdf_idx]
                    atcs_idx = unmatchedATCSarrivals[(unmatchedATCSarrivals["DATETIME"] == datetime_value)].index[0]
                    unmatchedATCSarrivals.drop(atcs_idx, axis=0, inplace=True)
                
                elif searchArrivals.iloc[0]["LINE"] == "X":
                    
                    # OPTION 4 -- MATCHED TO OUT-OF-SERVICE TRAIN
                    # the remaining, un-matched train in the subset search dataset is of line "X", which indicates 
                    # an out of service train. Because this is the only matching train within the time window and it does not contain 
                    # any information that would conflict with the information about the NextBus arrival, we are able to assert that 
                    # the NextBus arrival matches this train that the train control system considers to be out-of-service.
                    
                    matchCodeID = 4
                    print(arrivalVehID, "MATCHED -- Only eligible match is an out of service train")
                    #matchedNBidx.append(i)
                    unmatchedNBarrivals.drop(i, axis=0, inplace=True)
                    
                    sdf_idx = 0
                    datetime_value = searchArrivals["DATETIME"][sdf_idx]
                    atcs_idx = unmatchedATCSarrivals[(unmatchedATCSarrivals["DATETIME"] == datetime_value)].index[0]
                    unmatchedATCSarrivals.drop(atcs_idx, axis=0, inplace=True)
                    
                else:
                    
                    # OPTION 5 -- ONE REMAINING MATCH; DIFFERENT LINES 
                    # the remaining, un-matched train in the subset search dataset is of a different line than the line of 
                    # the NextBus arrival being investigated.
                    
                    matchCodeID = 5
                    print(arrivalVehID, "One remaining match; mismatched lines:", arrivalLine, "vs.", searchArrivals.iloc[0]["LINE"])
    
            
            # if there is more than one remaining, un-matched arrival record in the subset search dataset, there are a number of
            # steps we can take to try to match those search dataset records with the NextBus record we're investigating
            
            elif len(searchArrivals) > 1:
                try:
                    # count the number of times the line of the NextBus arrival appears in the un-matched subset search DataFrame
                    obs = dict(searchDFarrivals['LINE'].value_counts())[arrivalLine]
                    
                except KeyError:
                    # if the line of the NextBus arrival being investigated is not present in the subset search arrival records
                    # we first try to determine if there is a train considered to be out-of-service present
                    try:
                        if dict(searchDFarrivals['LINE'].value_counts())["X"] == 1:
                            
                            # OPTION 4 -- MATCHED TO AN OUT-OF-SERVICE TRAIN
                            # if there are no arrival records from the un-matched, subset search dataset that match the line of the 
                            # NextBus arrival being investigated, but there is one out-of-service train in that subset, we can match
                            # that out-of-service train to the NextBus arrival. This NextBus arrival is therefore considered to be MATCHED.
                            
                            matchCodeID = 4
                            obs = 0
                            print(arrivalVehID, "MATCHED -- MATCHED TO A OUT-OF-SERVICE TRAIN")
                            #matchedNBidx.append(i)
                            unmatchedNBarrivals.drop(i, axis=0, inplace=True)
                            
                            sdf_idx = list(searchDFarrivals["SECOND_CONSIST"]).index(arrivalVehID)
                            datetime_value = searchDFarrivals["DATETIME"][sdf_idx]
                            atcs_idx = unmatchedATCSarrivals[(unmatchedATCSarrivals["DATETIME"] == datetime_value) & (unmatchedATCSarrivals["SECOND_CONSIST"] == arrivalVehID)].index[0]
                            unmatchedATCSarrivals.drop(atcs_idx, axis=0, inplace=True)
                            
                        elif dict(searchDFarrivals['LINE'].value_counts())["X"] > 1:
                            
                            # OPTION 6 -- UNMATCHED-UNKNOWN; MULTIPLE OUT-OF-SERVICE TRAINS
                            # if there are no arrival records from the un-matched, subset search dataset that match the line of the 
                            # NextBus arrival being investigated, but there is more than one out-of-service trains in that subset, we are
                            # unable to make a match across datasets. This NextBus arrival is therefore considered to be UNMATCHED-UNKNOWN.
                            
                            matchCodeID = 6
                            obs = 0
                            print(arrivalVehID, "UNMATCHED-UNKNOWN -- multiple out-of-service trains")
                            
                        else:
                            
                            # this scenario is impossible; if there are 0 records with line "X", the above IF statement will throw
                            # a KeyError
                            pass
                        
                    except KeyError:
                        obs = 0
                        
                        # OPTION 10 -- UNMATCHED-UNKNOWN; MULTIPLE RECORDS OF OTHER LINES
                        # of the remaining, un-matched records, there are multiple records belonging to other lines
                        # than the line indidicated in the NextBus data
                        
                        matchCodeID = 10
                        print(arrivalVehID, "UNMATCHED-UNKNOWN - No records of NextBus line or out-of-service trains. Multiple records of other lines.")
                
                if obs == 1:
                    
                    # OPTION 7 -- MATCHED; ONE UN-MATCHED RECORD HAS SAME LINE
                    # of the remaining, un-matched records from the subset search dataset, only one belongs to the same
                    # line as the NextBus arrival being investigated, which allows us to match the NextBus arrival to that search 
                    # record. This NextBus arrival is therefore considered to be MATCHED.
                    
                    matchCodeID = 7
                    print(arrivalVehID, "MATCHED --  via removal & line (only one match with predicted line.)")
                    matches += 1
                    #matchedNBidx.append(i)
                    unmatchedNBarrivals.drop(i, axis=0, inplace=True)
                    
                    sdf_idx = list(searchDFarrivals["SECOND_CONSIST"]).index(arrivalVehID)
                    datetime_value = searchDFarrivals["DATETIME"][sdf_idx]
                    atcs_idx = unmatchedATCSarrivals[(unmatchedATCSarrivals["DATETIME"] == datetime_value) & (unmatchedATCSarrivals["SECOND_CONSIST"] == arrivalVehID)].index[0]
                    unmatchedATCSarrivals.drop(atcs_idx, axis=0, inplace=True)
                    
                elif obs > 1:
                    
                    # OPTION 8 -- UNMATCHED-UNKNOWN; MORE THAN 1 UNMATCHED ARRIVAL OF THAT LINE IN TIME-WINDOW
                    # if there is more than one un-matched arrival record from the subset search dataset that belongs to
                    # the same line as the NextBus arrival being investigated, we are unable to to match the NextBus arrival to an
                    # arrival in the subset search dataset. This NextBus arrival is therefore considered to be UNMATCHED-UNKNOWN.
                    
                    matchCodeID = 8
                    print(arrivalVehID, "UNMATCHED-UNKNOWN -- More than 1 unmatched arrival of that line in the time window.")
                
                else:
                    # OPTION 6 or OPTION 7 occurred and has been already noted.
                    pass
            else:
                # we can't have a DataFrame searchArrivals of negative length, so this scenario is impossible
                pass
    else:
        ## OPTION 9 -- GHOST TRAIN; NO RECORDS IN TIME-WINDOW 
        # no arrival records were present in the subset search dataset. 
        # This NextBus arrival is therefore considered to be a GHOST TRAIN.
        confirmedGhosts += 1
        matchCodeID = 9
        print(arrivalVehID, "GHOST TRAIN -- no arrivals in time window.")
        
    matchresults = [arrivalVehID, arrivalLine, toReadableTime(arrivalTime), matchCodeID, matchCodes[matchCodeID], altMatrix]
    arrivalMatching = arrivalMatching.append(pd.DataFrame([matchresults], columns=arrivalMatchingCols), ignore_index=True)  
    
for u in unmatchedNBarrivals.index:
    
    arrivalTime = arrivals.loc[u, "Timestamp"]
    arrivalVehID = arrivals.loc[u, "Vehicle"]
    arrivalLine = arrivals.loc[u, "Line"]
    arrivalReadableTime = arrivals.loc[u, "Arrival Time"]
    t_max = arrivalTime + timeRadius
    t_min = arrivalTime - 60
    
    # this matrix stores potential matches if more than one is found
    altMatrix = np.NaN
    
    # generate DFs of only arrivals within the time window across both datasets
    unmatched_searchDFarrivals = vnr_t[(vnr_t["UNIX"] < t_max) & (vnr_t["UNIX"] > t_min)]
    #print(unmatched_searchDFarrivals)
    
    if len(unmatched_searchDFarrivals[(unmatched_searchDFarrivals["FOUR_DIGIT_CONSIST"] == arrivalVehID)]) == 1:
        
        matchCodeID = 11
        
        sdf_idx = list(unmatched_searchDFarrivals["FOUR_DIGIT_CONSIST"]).index(arrivalVehID)
        datetime_value = unmatched_searchDFarrivals["DATETIME"].iloc[sdf_idx]
        atcs_idx = vnr_t[(vnr_t["DATETIME"] == datetime_value) & (vnr_t["FOUR_DIGIT_CONSIST"] == arrivalVehID)].index[0]

        matchTime = vnr_t.loc[atcs_idx, "TIME"]
        
        if vnr_t.loc[atcs_idx, "SECOND_CONSIST"] == 0:
            matchVeh = str(vnr_t.loc[atcs_idx, "FOUR_DIGIT_CONSIST"])
        else:
            matchVeh = str(vnr_t.loc[atcs_idx, "FOUR_DIGIT_CONSIST"]) + "/" + str(vnr_t.loc[atcs_idx, "SECOND_CONSIST"])

        matchLine = signs2consists[(signs2consists["CONSIST"] == vnr_t.loc[atcs_idx, "FOUR_DIGIT_CONSIST"]) & (signs2consists["UNIX"] < vnr_t.loc[atcs_idx, "UNIX"])].sort_values("UNIX").reset_index().tail(1)["LINE"].iloc[0]

        if matchLine in ["K","T"]:
            matchLine = "KT"
        else:
            pass

        matchPairs = [arrivalReadableTime, arrivalVehID, arrivalLine,matchTime,matchVeh,matchLine,matchCodes[matchCodeID]]
        matchDF = matchDF.append(pd.DataFrame([matchPairs], columns=matchDFcolumns), ignore_index=True)
        
        matchresults = [arrivalVehID, arrivalLine, toReadableTime(arrivalTime), matchCodeID, matchCodes[matchCodeID], altMatrix]
        arrivalMatching = arrivalMatching.append(pd.DataFrame([matchresults], columns=arrivalMatchingCols), ignore_index=True)
    
        unmatchedNBarrivals.drop(u, axis=0, inplace=True)
    
    elif len(unmatched_searchDFarrivals[(unmatched_searchDFarrivals["SECOND_CONSIST"] == arrivalVehID)]) == 1:
        
        matchCodeID = 11
        
        sdf_idx = list(unmatched_searchDFarrivals["SECOND_CONSIST"]).index(arrivalVehID)
        datetime_value = unmatched_searchDFarrivals["DATETIME"].iloc[sdf_idx]
        atcs_idx = vnr_t[(vnr_t["DATETIME"] == datetime_value) & (vnr_t["SECOND_CONSIST"] == arrivalVehID)].index[0]

        matchTime = vnr_t.loc[atcs_idx, "TIME"]
        
        if vnr_t.loc[atcs_idx, "SECOND_CONSIST"] == 0:
            matchVeh = str(vnr_t.loc[atcs_idx, "FOUR_DIGIT_CONSIST"])
        else:
            matchVeh = str(vnr_t.loc[atcs_idx, "FOUR_DIGIT_CONSIST"]) + "/" + str(vnr_t.loc[atcs_idx, "SECOND_CONSIST"])

        matchLine = signs2consists[(signs2consists["CONSIST"] == vnr_t.loc[atcs_idx, "FOUR_DIGIT_CONSIST"]) & (signs2consists["UNIX"] < vnr_t.loc[atcs_idx, "UNIX"])].sort_values("UNIX").reset_index().tail(1)["LINE"].iloc[0]

        if matchLine in ["K","T"]:
            matchLine = "KT"
        else:
            pass

        matchPairs = [arrivalReadableTime, arrivalVehID, arrivalLine,matchTime,matchVeh,matchLine,matchCodes[matchCodeID]]
        matchDF = matchDF.append(pd.DataFrame([matchPairs], columns=matchDFcolumns), ignore_index=True)

        matchresults = [arrivalVehID, arrivalLine, toReadableTime(arrivalTime), matchCodeID, matchCodes[matchCodeID], altMatrix]
        arrivalMatching = arrivalMatching.append(pd.DataFrame([matchresults], columns=arrivalMatchingCols), ignore_index=True)
    
        unmatchedNBarrivals.drop(u, axis=0, inplace=True)

2020.0 GHOST TRAIN -- no remaining, un-matched arrival in time window.
2028.0 GHOST TRAIN -- no remaining, un-matched arrival in time window.
1414.0 GHOST TRAIN -- no remaining, un-matched arrival in time window.
1526.0 GHOST TRAIN -- no remaining, un-matched arrival in time window.
1469.0 GHOST TRAIN -- no remaining, un-matched arrival in time window.
1528.0 GHOST TRAIN -- no remaining, un-matched arrival in time window.
1458.0 GHOST TRAIN -- no remaining, un-matched arrival in time window.
1484.0 GHOST TRAIN -- no arrivals in time window.
1523.0 GHOST TRAIN -- no arrivals in time window.
1518.0 GHOST TRAIN -- no remaining, un-matched arrival in time window.
1476.0 GHOST TRAIN -- no remaining, un-matched arrival in time window.
1521.0 GHOST TRAIN -- no remaining, un-matched arrival in time window.
1428.0 One remaining match; mismatched lines: N vs. S
1496.0 One remaining match; mismatched lines: KT vs. J
1478.0 GHOST TRAIN -- no arrivals in time window.


In [16]:
matchDF["Line Match"] = matchDF["NextBus Line"] == matchDF["ATCS Line"]
matchDF = matchDF.sort_values("NextBus Time").reset_index(drop=True)
matchDF

,NextBus Time,NextBus Vehicle,NextBus Line,ATCS Time,ATCS Vehicle,ATCS Line,Match Type,Line Match
0,00:04:26,1412.0,KT,00:04:32,1412,KT,MATCHED ON VEHICLE ID,True
1,00:06:08,2037.0,N,00:06:23,2037/2004,N,MATCHED ON VEHICLE ID,True
2,00:07:53,1499.0,M,00:08:07,1499/1511,M,MATCHED ON VEHICLE ID,True
3,00:09:06,1516.0,L,00:09:24,1516/1456,L,MATCHED ON VEHICLE ID,True
4,00:20:53,1468.0,N,00:21:12,1468/1482,N,MATCHED ON VEHICLE ID,True
5,00:22:43,1525.0,M,00:22:58,1525/1480,M,MATCHED ON VEHICLE ID,True
6,00:24:11,1448.0,J,00:24:29,1448,J,MATCHED ON VEHICLE ID,True
7,00:33:20,1479.0,L,00:33:39,1479/1413,L,MATCHED ON VEHICLE ID,True
8,00:37:24,1509.0,N,00:37:40,1509/1408,N,MATCHED ON VEHICLE ID,True
9,00:40:02,1508.0,N,00:40:20,1508/1424,N,MATCHED ON VEHICLE ID,True


## 6. Match Remaining ATCS to NextBus

This section includes the "Back Matcher". which iterates through unmatched ATCS observations and tries to match them to unmatched NextBus predicted arrivals. The work flow is similar to that of the preceeding section: if a match is determined to have occurred across datasets, the NextBus predicted arrival is removed from _unmatchedNBarrivals_, and the ATCS observation is removed from _unmatchedATCSarrivals_.



In [17]:
# iterate through remaining unmatched ATCS observations
for j in unmatchedATCSarrivals.index:
    
    # attributes of the unmatched ATCS observation
    arrivalUnix = unmatchedATCSarrivals.loc[j, "UNIX"]
    four_digit_consist = unmatchedATCSarrivals.loc[j, "FOUR_DIGIT_CONSIST"]
    second_consist = unmatchedATCSarrivals.loc[j, "SECOND_CONSIST"]
    
    # assign line information to the unmatched ATCS observation
    subsetLines = signs2consists[(signs2consists["CONSIST"].isin([four_digit_consist, second_consist])) & (signs2consists["UNIX"] <= arrivalUnix) & (signs2consists["UNIX"] >= (arrivalUnix - (60*60)))].sort_values("UNIX")
    ATCSline = subsetLines.loc[subsetLines["UNIX"].idxmax(), "LINE"]
    
    if ATCSline == "T":
        ATCSline = "KT"
    else:
        pass
    
    # create a DataFrame of unmatched NextBus arrivals within the time window surrounding the the ATCS arrival
    potentialBackMatches = unmatchedNBarrivals[(unmatchedNBarrivals["Arrival Timestamp"] < (arrivalUnix + timeRadius)) & (unmatchedNBarrivals["Arrival Timestamp"] > (arrivalUnix - timeRadius))].copy(deep=True)
    potentialBackMatches["ArrivalTimeDif"] = [x - arrivalUnix for x in potentialBackMatches["Arrival Timestamp"]]
    potentialBackMatches["PercentileTimeDif"] = [stats.percentileofscore(matchTimeDif, x) for x in potentialBackMatches["ArrivalTimeDif"]]
    
    # if there is a potential match, iterate through
    if len(potentialBackMatches) != 0:
        for l in potentialBackMatches.index:
            
            # check if there is a line match
            if ATCSline == potentialBackMatches.loc[l, "Line"]:
                
                matchCodeID = 7
                matchresults = [potentialBackMatches.loc[l, "Vehicle"], potentialBackMatches.loc[l, "Line"], toReadableTime(potentialBackMatches.loc[l, "Timestamp"]), matchCodeID, matchCodes[matchCodeID], np.NaN]
                arrivalMatching = arrivalMatching.append(pd.DataFrame([matchresults], columns=arrivalMatchingCols), ignore_index=True)  

                unmatchedNBarrivals.drop(l, axis=0, inplace=True)
            else:
                pass
    else:
        pass
                
        
arrivalMatching.drop_duplicates(subset=["NextBus Vehicle ID", "NextBus Arrival Time"], keep="last", inplace=True)

After both the Matcher and the Back Matcher are complete and the matching process is over, assign lines to unmatched ATCS records for final report writing.

In [18]:
sign_dfs = []

for n in unmatchedATCSarrivals.index:
    car1 = unmatchedATCSarrivals.loc[n, "FOUR_DIGIT_CONSIST"]
    unix = unmatchedATCSarrivals.loc[n, "UNIX"]

    #print(car1, unix)

    viableSignOptions = signs2consists[(signs2consists["CONSIST"].isin([car1])) & (signs2consists["UNIX"] <= unix) & (signs2consists["UNIX"] >= (unix - (60*60)))].sort_values("UNIX")
    #print(viableSignOptions)
    if len(tempDF) == 0:
        line = "X"
    else:
        line = viableSignOptions.loc[viableSignOptions["UNIX"].idxmax(), "LINE"]
    #print(line)
    sign_dfs.append(pd.DataFrame([[car1, unix, line]], columns=["FOUR_DIGIT_CONSIST", "UNIX", "LINE"]))

try:
    unmatchedATCSLineAssignments = pd.concat(sign_dfs, ignore_index=True)
except ValueError:
    print(len(searchDFarrivals))

unmatchedATCSarrivals = unmatchedATCSarrivals.merge(unmatchedATCSLineAssignments, on=["FOUR_DIGIT_CONSIST","UNIX"])

## 7. Write Report

The final section of this notebook writes a summary of the day's matches to a .txt file.

In [19]:
descriptiveStats = pd.DataFrame(arrivalMatching["Match Code Description"].value_counts())

# calculate total number of matched NextBus Arrivals
totalNBmatches = descriptiveStats.loc["MATCHED ON VEHICLE ID", "Match Code Description"]
if 'MATCHED; ONE REMAINING MATCH; SAME LINE' in descriptiveStats.index:
    totalNBmatches += descriptiveStats.loc['MATCHED; ONE REMAINING MATCH; SAME LINE', "Match Code Description"]
if 'MATCHED; ONE UN-MATCHED RECORD HAS SAME LINE' in descriptiveStats.index:
    totalNBmatches += descriptiveStats.loc['MATCHED; ONE UN-MATCHED RECORD HAS SAME LINE', "Match Code Description"]
if 'MATCHED TO OUT-OF-SERVICE TRAIN' in descriptiveStats.index:
    totalNBmatches += descriptiveStats.loc['MATCHED TO OUT-OF-SERVICE TRAIN', "Match Code Description"]

print(totalNBmatches, "total matches")

284 total matches


In [21]:
reportPath = "test-REPORT-{}.txt".format(day2read)

f = open(reportPath, "w")

f.write("NextBus/ATCS Matching Report for " + day2read + "\n")
f.write("For support, contact Transit Technology Group.\n")
f.write("\n")
f.write("** Daily Overview for " + day2read + " **\n")
f.write("\n")

# summary messages to write at the top of the report
nb_total_msg = "{} NextBus Arrivals - {} Matched ({}%)\n".format(len(arrivalMatching), totalNBmatches, str(int(round((totalNBmatches/len(arrivalMatching)*100)))))
atcs_total_msg = "{} ATCS Arrivals - {} Matched ({}%)\n".format(len(vnr_t), totalNBmatches, str(int(round((totalNBmatches/len(vnr_t)*100)))))

f.write(nb_total_msg)
f.write(atcs_total_msg)
f.write("\n")

for i in descriptiveStats.index:
    
    count = int(descriptiveStats.loc[i, "Match Code Description"])
    pct = str(round(count/len(arrivalMatching),3) * 100)
    
    beginning = i
    end = "{} of {} arrivals ({}%)".format(count, len(arrivalMatching), pct)
    
    summary = beginning + " " + ''.join(["-" for x in np.zeros(108-(len(beginning) + len(end)))]) + " " + end + "\n"
    
    f.write(summary)

f.write("\n\n")
f.write("Vehicle IDs listed below are considered to be un-matched. They are sorted by matching failure.\n\n")
#f.write("Arrivals are shown in the following format:\n<NEXTBUS ARRIVAL PREDICTION> -- <VEHICLE ID>\n\n")

for i in list(descriptiveStats[descriptiveStats.index.isin([matchCodes[x] for x in[2,5,6,8,9,10]])].index):
    counter = 1
    f.write("* " + i + " * \n")
    f.write(matchDescriptions[i] + "\n\n")
    #f.write("Arrival Time -- Vehicle ID\n")
    preheader = "---------NB------------"
    header = "  TIME         ID     |"
    if i in [matchCodes[x] for x in [5,8,10]]:
        preheader += "------------ATCS------------"
        header += "  TIME           ID        |"
    preheader += "\n"
    header += "\n-----------------------\n"
    f.write(preheader)
    f.write(header)
    for row in arrivalMatching[arrivalMatching["Match Code Description"] == i].iterrows():
        index, contents = row

        vehID = contents[0]
        vehLine = contents[1]
        arrivalTimestamp = contents[2]
        vehAltMatrix = contents[5]
        toWrite = "{}. ".format(str(counter))
        toWrite += "{0: <21}".format(str(arrivalTimestamp) + " -- " + str(vehID) + " (" + vehLine + ")")
        if i in [matchCodes[x] for x in [5,8,10]]:
            for j in vehAltMatrix:
                if j[0,4] == 0:
                    consist = str(j[0,3])
                else:
                    consist = str(j[0,3]) +"/" + str(j[0,4])
                toWrite += "{0: <27}".format(" | " + str(j[0,1]) + " -- " + str(consist) + " (" + str(j[0,5]) + ")")
        toWrite += "\n"
        f.write(toWrite)
        counter +=1
    f.write("\n")
f.write("\n\n")

# ADD IN UNMATCHED ATCS VALUES

f.write("* ATCS ARRIVALS WITHOUT CORRESPONDING NEXTBUS MATCHES *\n")
counter = 1
for k in unmatchedATCSarrivals.index:
    aTime = unmatchedATCSarrivals.loc[k, "TIME"]
    
    aLine = unmatchedATCSarrivals.loc[k, "LINE"]
    
    if aLine == "K" or aLine == "T":
        aLine = "KT"
    else:
        pass
    
    if unmatchedATCSarrivals.loc[k, "SECOND_CONSIST"] != 0:
        aConsist = str(unmatchedATCSarrivals.loc[k, "FOUR_DIGIT_CONSIST"]) +"/" +str(unmatchedATCSarrivals.loc[k, "SECOND_CONSIST"])
    else:
        aConsist = str(unmatchedATCSarrivals.loc[k, "FOUR_DIGIT_CONSIST"])
    toWrite = "{}. {} -- {} ({})\n".format(str(counter),aTime,aConsist,aLine)
    f.write(toWrite)
    counter += 1
f.close()

print("Script has run successfully.")

Script has run successfully.
